In [ ]:
from qdrant_client import QdrantClient
from qdrant_client.models import VectorParams, PointStruct
from sentence_transformers import SentenceTransformer

# Connect to local Qdrant container
client = QdrantClient(host="localhost", port=6333)

# Create collection
collection_name = "demo_collection"
client.recreate_collection(
    collection_name=collection_name,
    vectors_config=VectorParams(size=384, distance="Cosine")
)

# Load model
model = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")

# Sample data
texts = [
    "Machine learning enables computers to learn from data.",
    "Vector databases store high-dimensional embeddings efficiently.",
    "Qdrant makes it easy to perform semantic search."
]

# Generate embeddings
embeddings = model.encode(texts)

# Insert points
points = [
    PointStruct(id=i, vector=embeddings[i].tolist(), payload={"text": texts[i]})
    for i in range(len(texts))
]
client.upsert(collection_name=collection_name, points=points)

# Verify
result = client.scroll(collection_name=collection_name, limit=3)
for point in result[0]:
    print(point.payload)
